In [ ]:
import cv2
import torch
import torch.nn as nn
from torchvision import models, transforms
from efficientnet_pytorch import EfficientNet
from ultralytics import YOLO
from PIL import Image
import numpy as np


### Device and Transforms

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                         std=[0.229, 0.224, 0.225])
])

### Load Models

In [ ]:
resnet = models.resnet18(pretrained=False)
resnet.fc = nn.Sequential(
    nn.Linear(512, 128),
    nn.ReLU(),
    nn.Dropout(0.4),
    nn.Linear(128, 2)
)
resnet.load_state_dict(torch.load("resnet_facial_hair_binary.pt"))
resnet.to(device).eval()

effnet = EfficientNet.from_name('efficientnet-b0')
effnet._fc = nn.Linear(effnet._fc.in_features, 2)
effnet.load_state_dict(torch.load("efficientnet_facial.pt"))
effnet.to(device).eval()

yolo_model = YOLO("best_hairnet.pt")

### Prediction Functions

In [ ]:
def predict_facial_hair(img, alpha=0.6):
    img_tensor = transform(img).unsqueeze(0).to(device)
    with torch.no_grad():
        r_out = torch.softmax(resnet(img_tensor), dim=1)
        e_out = torch.softmax(effnet(img_tensor), dim=1)
        combined = (1 - alpha) * r_out + alpha * e_out
        _, pred = torch.max(combined, 1)
    return pred.item()  # 0 = No Beard, 1 = Beard

def detect_hairnet_from_np(img_np):
    img_bgr = cv2.cvtColor(img_np, cv2.COLOR_RGB2BGR)
    results = yolo_model.predict(img_bgr, conf=0.25, save=False, verbose=False)
    names = yolo_model.names
    detections = results[0].boxes.cls.cpu().numpy() if results[0].boxes else []
    detected_classes = [names[int(cls)] for cls in detections]
    return 'hairnet' in detected_classes

### Webcam Compliance Checker

In [ ]:
cap = cv2.VideoCapture(0)
font = cv2.FONT_HERSHEY_SIMPLEX

print("Press 'q' to quit.")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Convert to PIL Image
    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    img_pil = Image.fromarray(img_rgb)

    # Predictions
    beard = predict_facial_hair(img_pil) == 1
    hairnet = detect_hairnet_from_np(img_rgb)
    compliant = not beard and hairnet

    # Message
    beard_text = "Beard: Yes" if beard else "Beard: No"
    hairnet_text = "Hairnet: Yes" if hairnet else "Hairnet: No"
    final_status = "COMPLIANT" if compliant else "NON-COMPLIANT"

    # Draw overlay
    overlay = frame.copy()
    cv2.putText(overlay, beard_text, (10, 30), font, 0.8, (0, 255, 255), 2)
    cv2.putText(overlay, hairnet_text, (10, 60), font, 0.8, (0, 255, 255), 2)
    cv2.putText(overlay, final_status, (10, 100), font, 1.0, (0, 0, 255) if not compliant else (0, 255, 0), 3)

    # Display
    cv2.imshow('Hygiene Compliance Checker', overlay)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()